In [1]:
import os
os.environ['THEANO_FLAGS'] = "device=gpu"  

In [2]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
import keras.backend as K

TRAIN_DIR = 'Dataset/8_frames/train'
VALID_DIR = 'Dataset/8_frames/validation'


SIZE = (224, 224)
BATCH_SIZE = 32


if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)]) #train data 수집
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)]) #validation data 수집

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE) 
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator() # Train을 위한 DataGenerator 사용
    val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True)  # validation을 위한 DataGenerator 사용

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)

    model = keras.applications.resnet50.ResNet50() #디
    

    
    classes = list(iter(batches.class_indices))
    model.layers.pop()
    
    for layer in model.layers:
        if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
            layer.trainable = True
            K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
            K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
        else:
            layer.trainable = False
            
    last = model.layers[-1].output
    x = Dense(len(classes), activation="softmax")(last)
    finetuned_model = Model(model.input, x)
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
    finetuned_model.classes = classes
    
    fileName="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    checkpointer = ModelCheckpoint(fileName, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    history = finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=1,validation_data=val_batches, validation_steps=num_valid_steps)
    finetuned_model.save('resnet50_8_camera.h5')


Using TensorFlow backend.
C:\Users\inha\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\inha\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\inha\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\inha\AppData\Roaming

Found 106512 images belonging to 6 classes.
Found 37464 images belonging to 6 classes.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
 161/3328 [>.............................] - ETA: 38:26 - loss: 1.3339 - acc: 0.4748

In [ ]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model
import time
start = time.time() 
caltech_dir = "Dataset/8_frames/test/pan"
image_w = 224
image_h = 224

total_acc = 0

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('resnet50_8_camera.h5')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_pan = 0
cnt1_1_pan = 0
cnt2_1_pan = 0
cnt3_1_pan = 0 
cnt4_1_pan = 0
cnt5_1_pan = 0
cnt6_1_pan = 0

pan_1_acc = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_pan +=1
    elif pre_ans == 1: 
        cnt1_1_pan +=1
    elif pre_ans == 2: 
        cnt2_1_pan +=1
    elif pre_ans == 3: 
        cnt3_1_pan +=1
    elif pre_ans == 4: 
        cnt4_1_pan +=1
    elif pre_ans == 5: 
        cnt5_1_pan +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_pan += 1


pan_1_acc = cnt0_1_pan/cnt6_1_pan
    
print("pan의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개 ,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_pan,cnt0_1_pan,cnt1_1_pan,cnt2_1_pan,cnt3_1_pan,cnt4_1_pan,cnt5_1_pan))
print("정확도는 %d 입니다."%(pan_1_acc))
print("time :", time.time() - start)


start = time.time() 
caltech_dir = "Dataset/8_frames/test/pedestal"
image_w = 224
image_h = 224

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_pedestal = 0
cnt1_1_pedestal = 0
cnt2_1_pedestal = 0
cnt3_1_pedestal = 0 
cnt4_1_pedestal = 0
cnt5_1_pedestal = 0
cnt6_1_pedestal = 0

pedestal_1_acc = 0


#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_pedestal +=1
    elif pre_ans == 1: 
        cnt1_1_pedestal +=1
    elif pre_ans == 2: 
        cnt2_1_pedestal +=1
    elif pre_ans == 3: 
        cnt3_1_pedestal +=1
    elif pre_ans == 4: 
        cnt4_1_pedestal +=1
    elif pre_ans == 5: 
        cnt5_1_pedestal +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_pedestal += 1

print("pedestal의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_pedestal,cnt0_1_pedestal,cnt1_1_pedestal,cnt2_1_pedestal,cnt3_1_pedestal,cnt4_1_pedestal,cnt5_1_pedestal))

pedestal_1_acc = cnt1_1_pedestal/cnt6_1_pedestal

print("정확도는 %d 입니다."%(pedestal_1_acc))
print("time :", time.time() - start)


start = time.time() 
caltech_dir = "Dataset/8_frames/test/tilt"
image_w = 224
image_h = 224

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_tilt = 0
cnt1_1_tilt = 0
cnt2_1_tilt = 0
cnt3_1_tilt = 0 
cnt4_1_tilt = 0
cnt5_1_tilt = 0
cnt6_1_tilt = 0

tilt_1_acc = 0


#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_tilt +=1
    elif pre_ans == 1: 
        cnt1_1_tilt +=1
    elif pre_ans == 2: 
        cnt2_1_tilt +=1
    elif pre_ans == 3: 
        cnt3_1_tilt +=1
    elif pre_ans == 4: 
        cnt4_1_tilt +=1
    elif pre_ans == 5: 
        cnt5_1_tilt +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_tilt += 1

print("tilt의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_tilt,cnt0_1_tilt,cnt1_1_tilt,cnt2_1_tilt,cnt3_1_tilt,cnt4_1_tilt,cnt5_1_tilt))

tilt_1_acc = cnt2_1_tilt/cnt6_1_tilt

print("정확도는 %d 입니다."%(tilt_1_acc))
print("time :", time.time() - start)


start = time.time() 
caltech_dir = "Dataset/8_frames/test/truck"
image_w = 224
image_h = 224

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_truck = 0
cnt1_1_truck = 0
cnt2_1_truck = 0
cnt3_1_truck = 0 
cnt4_1_truck = 0
cnt5_1_truck = 0
cnt6_1_truck = 0

truck_1_acc = 0


#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_truck +=1
    elif pre_ans == 1: 
        cnt1_1_truck +=1
    elif pre_ans == 2: 
        cnt2_1_truck +=1
    elif pre_ans == 3: 
        cnt3_1_truck +=1
    elif pre_ans == 4: 
        cnt4_1_truck +=1
    elif pre_ans == 5: 
        cnt5_1_truck +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_truck += 1

print("truck의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_truck,cnt0_1_truck,cnt1_1_truck,cnt2_1_truck,cnt3_1_truck,cnt4_1_truck,cnt5_1_truck))

truck_1_acc = cnt3_1_truck/cnt6_1_truck

print("정확도는 %d 입니다."%(truck_1_acc))
print("time :", time.time() - start)


start = time.time() 
caltech_dir = "Dataset/8_frames/test/zoom_in"
image_w = 224
image_h = 224

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_zoom_in = 0
cnt1_1_zoom_in = 0
cnt2_1_zoom_in = 0
cnt3_1_zoom_in = 0 
cnt4_1_zoom_in = 0
cnt5_1_zoom_in = 0
cnt6_1_zoom_in = 0

zoom_in_1_acc = 0


#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_zoom_in +=1
    elif pre_ans == 1: 
        cnt1_1_zoom_in +=1
    elif pre_ans == 2: 
        cnt2_1_zoom_in +=1
    elif pre_ans == 3: 
        cnt3_1_zoom_in +=1
    elif pre_ans == 4: 
        cnt4_1_zoom_in +=1
    elif pre_ans == 5: 
        cnt5_1_zoom_in +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_zoom_in += 1

print("zoom_in의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_zoom_in,cnt0_1_zoom_in,cnt1_1_zoom_in,cnt2_1_zoom_in,cnt3_1_zoom_in,cnt4_1_zoom_in,cnt5_1_zoom_in))

zoom_in_1_acc = cnt4_1_zoom_in/cnt6_1_zoom_in

print("정확도는 %d 입니다."%(zoom_in_1_acc))
print("time :", time.time() - start)



start = time.time() 
caltech_dir = "Dataset/8_frames/test/zoom_out"
image_w = 224
image_h = 224

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt0_1_zoom_out = 0
cnt1_1_zoom_out = 0
cnt2_1_zoom_out = 0
cnt3_1_zoom_out = 0 
cnt4_1_zoom_out = 0
cnt5_1_zoom_out = 0
cnt6_1_zoom_out = 0

zoom_out_1_acc = 0


#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    if pre_ans == 0: 
        cnt0_1_zoom_out +=1
    elif pre_ans == 1: 
        cnt1_1_zoom_out +=1
    elif pre_ans == 2: 
        cnt2_1_zoom_out +=1
    elif pre_ans == 3: 
        cnt3_1_zoom_out +=1
    elif pre_ans == 4: 
        cnt4_1_zoom_out +=1
    elif pre_ans == 5: 
        cnt5_1_zoom_out +=1
    #print("long으로",filenames[cnt3].split("\\")[1])
    cnt6_1_zoom_out += 1

print("zoom_out의 총 data 수 %d개 중 pan : %d개, pedestal %d개, tilt %d개, truck %d개,zoom_in %d개, zoom_out %d개 예측하였습니다"%(cnt6_1_zoom_out,cnt0_1_zoom_out,cnt1_1_zoom_out,cnt2_1_zoom_out,cnt3_1_zoom_out,cnt4_1_zoom_out,cnt5_1_zoom_out))

zoom_out_1_acc = cnt5_1_zoom_in/cnt6_1_zoom_in

print("정확도는 %d 입니다."%(zoom_out_1_acc))
print("time :", time.time() - start)

total_acc = (zoom_out_1_acc + zoom_in_1_acc + truck_1_acc + pan_1_acc + pedestal_1_acc + tilt_1_acc)/6

print("1프레임의 정확도는 %d입니다"%(total_acc))
